In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_dir = '/content/drive/MyDrive/IndoorOutdoorDataset'

In [ ]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='training',
)

Found 800 files belonging to 2 classes.
Using 640 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory=dataset_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(img_width, img_height),
    batch_size=batch_size,
    validation_split=0.2,
    seed=42,
    shuffle=True,
    subset='validation',
)

Found 800 files belonging to 2 classes.
Using 160 files for validation.


In [ ]:
model = models.Sequential()
model.add(InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3), classes=2)),
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu')),
model.add(layers.Dense(128, activation='relu')),
model.add(layers.Dense(1, activation='sigmoid'))


87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 256)               13107456  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 34943265 (133.30 MB)
Trainable params: 34908833 (133.17 MB)
Non-trainable params: 34432 (134.50 KB)
_________________________________________________________________


In [ ]:
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/100
20/20 [==============================] - 269s 6s/step - loss: 2.7387 - accuracy: 0.7016 - val_loss: 28.7908 - val_accuracy: 0.4500
Epoch 2/100
20/20 [==============================] - 7s 329ms/step - loss: 0.2628 - accuracy: 0.8891 - val_loss: 11.4078 - val_accuracy: 0.5437
Epoch 3/100
20/20 [==============================] - 6s 276ms/step - loss: 0.1696 - accuracy: 0.9531 - val_loss: 11.7690 - val_accuracy: 0.5312
Epoch 4/100
20/20 [==============================] - 7s 298ms/step - loss: 0.0897 - accuracy: 0.9656 - val_loss: 29.2420 - val_accuracy: 0.4187
Epoch 5/100
20/20 [==============================] - 8s 362ms/step - loss: 0.1020 - accuracy: 0.9703 - val_loss: 87.1721 - val_accuracy: 0.5000
Epoch 6/100
20/20 [==============================] - 7s 276ms/step - loss: 0.3164 - accuracy: 0.9641 - val_loss: 13102013.0000 - val_accuracy: 0.5063
Epoch 7/100
20/20 [==============================] - 7s 295ms/step - loss: 0.1981 - accuracy: 0.9359 - val_loss: 76234296.0000 - va

In [ ]:
model.save('/content/drive/MyDrive/InceptionV3.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
